In [20]:
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected.")
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from scipy import io
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

mnist_data = []
spam_data = []
cifar_data = []
all_data = []

for data_name in ["mnist", "spam", "cifar10"]:
    data = io.loadmat("data/%s_data.mat" % data_name)
    all_data.append(data)
    print("\nloaded %s data!" % data_name)
    fields = "test_data", "training_data", "training_labels"
    for field in fields:
        print(field, data[field].shape)

mnist_data = all_data[0]
spam_data = all_data[1]
cifar_data = all_data[2]

mnist_v = mnist_data["training_data"][:10000]
y_mnist_v = mnist_data["training_labels"][:10000]
mnist_v, y_mnist_v = shuffle(mnist_v, y_mnist_v) # val set

mnist_t = mnist_data["training_data"]
y_mnist_t = mnist_data["training_labels"]

spam_t = spam_data['training_data']
y_spam_t =  spam_data['training_labels']
spam_t, y_spam_t = shuffle(spam_t, y_spam_t) #train set

spam_v = spam_t[:1034]
y_spam_v = y_spam_t[:1034]

spam_t = spam_t[1034:]
y_spam_t = y_spam_t[1034:]

cifar_v = cifar_data["training_data"][:5000]
y_cifar_v = cifar_data["training_labels"][:5000]
cifar_v, y_cifar_v = shuffle(cifar_v, y_cifar_v) # val set

cifar_t = cifar_data["training_data"]
y_cifar_t = cifar_data["training_labels"]
#cifar_t, y_cifar_t = shuffle(cifar_t, y_cifar_t) # train set



loaded mnist data!
test_data (10000, 784)
training_data (60000, 784)
training_labels (60000, 1)

loaded spam data!
test_data (5857, 32)
training_data (5172, 32)
training_labels (5172, 1)

loaded cifar10 data!
test_data (10000, 3072)
training_data (50000, 3072)
training_labels (50000, 1)


In [25]:
min_max_scaler = preprocessing.MinMaxScaler() # scale in [0,1]
X_t = min_max_scaler.fit_transform(mnist_t)
X_v = min_max_scaler.fit_transform(mnist_v)
samps = [100, 200, 500, 1000, 2000, 5000, 10000]
svm = LinearSVC()
for s in samps:
    svm.fit(X_t[:s], y_mnist_t[:s].ravel())
    y_p = svm.predict(X_v)
    print(accuracy_score(y_mnist_v, y_p))


0.7094
0.761
0.8286
0.8559
0.8831
0.932
0.9681


In [4]:
samps = [100, 200, 500, 1000, 2000, 4138]
svm_spam = LinearSVC()
for s in samps:
    svm_spam.fit(spam_t[:s], y_spam_t[:s].ravel())
    y_p = svm_spam.predict(spam_v)
    print(accuracy_score(y_spam_v, y_p))

0.8036750483558994
0.7485493230174082
0.7862669245647969
0.7959381044487428
0.8075435203094777
0.8065764023210832


In [5]:
samps = [100, 200, 500, 1000, 2000,5000]
svm_cifar = LinearSVC()

for s in samps:
    svm_cifar.fit(cifar_t[:s], y_cifar_t[:s].ravel())
    y_p = svm_cifar.predict(cifar_v)
    print(accuracy_score(y_cifar_v, y_p))

0.207
0.259
0.3224
0.3962
0.5452
0.8308


In [6]:
c_vals = [0.01, 0.021, 0.1, 0.31, 6, 50, 100, 500]
samps = [10000]
for c in c_vals:
    svm = LinearSVC(C=c)
    print('Accuracy for C param:'  + str(c))
    for s in samps:
        svm.fit(X_t[:s], y_mnist_t[:s].ravel())
        y_p = svm.predict(X_v)
        
        print(accuracy_score(y_mnist_v, y_p))


Accuracy for C param:0.01
0.9288
Accuracy for C param:0.021
0.9368
Accuracy for C param:0.1
0.9507
Accuracy for C param:0.31
0.9594
Accuracy for C param:6
0.9732
Accuracy for C param:50
0.9586
Accuracy for C param:100
0.9575
Accuracy for C param:500
0.9628


In [3]:
Xt_spam = spam_data['training_data']
yt_spam =  spam_data['training_labels']
Xt_spam, yt_spam = shuffle(Xt_spam, yt_spam) #train set

parts_x = []
parts_y = []
c_vals = [0.01, 0.021, 0.1, 0.31, 6, 50, 100, 500]

for i in range(1,6):
    
    parts_x.append(Xt_spam[(i-1)*1034:i*1034])
    parts_y.append(yt_spam[(i-1)*1034:i*1034])
acc = []
for c in c_vals:
    svm_spam = LinearSVC(C = c)
    for i in range (0,5):
        j = i
        if j == 0:
            svm_spam.fit(parts_x[j+4], parts_y[j+4].ravel())
            svm_spam.fit(parts_x[j+3], parts_y[j+3].ravel())
            svm_spam.fit(parts_x[j+2], parts_y[j+2].ravel())
            svm_spam.fit(parts_x[j+1], parts_y[j+1].ravel())
            y_p = svm_spam.predict(parts_x[i])
            acc.append((accuracy_score(parts_y[i], y_p)))

        if j == 1:
            svm_spam.fit(parts_x[j+3], parts_y[j+3].ravel())
            svm_spam.fit(parts_x[j+2], parts_y[j+2].ravel())
            svm_spam.fit(parts_x[j+1], parts_y[j+1].ravel())
            svm_spam.fit(parts_x[j-1], parts_y[j-1].ravel())
            y_p = svm_spam.predict(parts_x[i])
            acc.append((accuracy_score(parts_y[i], y_p)))

        if j == 2:
            svm_spam.fit(parts_x[j-2], parts_y[j-2].ravel())
            svm_spam.fit(parts_x[j-1], parts_y[j-1].ravel())
            svm_spam.fit(parts_x[j+1], parts_y[j+1].ravel())
            svm_spam.fit(parts_x[j+2], parts_y[j+2].ravel())
            y_p = svm_spam.predict(parts_x[i])
            acc.append((accuracy_score(parts_y[i], y_p)))

        if j == 3:
            svm_spam.fit(parts_x[j-3], parts_y[j-3].ravel())
            svm_spam.fit(parts_x[j-2], parts_y[j-2].ravel())
            svm_spam.fit(parts_x[j-1], parts_y[j-1].ravel())
            svm_spam.fit(parts_x[j+1], parts_y[j+1].ravel())
            y_p = svm_spam.predict(parts_x[i])
            acc.append((accuracy_score(parts_y[i], y_p)))

        if j == 4:
            svm_spam.fit(parts_x[j-4], parts_y[j-4].ravel())
            svm_spam.fit(parts_x[j-3], parts_y[j-3].ravel())
            svm_spam.fit(parts_x[j-2], parts_y[j-2].ravel())
            svm_spam.fit(parts_x[j-1], parts_y[j-1].ravel())
            y_p = svm_spam.predict(parts_x[i])
            acc.append((accuracy_score(parts_y[i], y_p)))
    print("Accuracy scores at c="+ str(c), np.average(acc))
    acc = []
    
#use 5 fold cross val


Accuracy scores at c=0.01 0.7845261121856867
Accuracy scores at c=0.021 0.7914893617021276
Accuracy scores at c=0.1 0.8003868471953579
Accuracy scores at c=0.31 0.8017408123791103
Accuracy scores at c=6 0.8063829787234044
Accuracy scores at c=50 0.8058027079303676
Accuracy scores at c=100 0.8013539651837525
Accuracy scores at c=500 0.7793036750483558
